# Advent of Code 2021

I really liked [Peter Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) to writing and tracking solutions, so I'm going to use his method this year.

## Day 0: Imports and Utility Functions

Preparations prior to Day 1:

- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.


In [71]:
from collections import namedtuple
from math import prod
import operator
from typing import Dict, Tuple, Set, List, Iterator, Optional, Union

In [20]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2021/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got


In [46]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def sliding_window(iterable, window_size) -> list:
    for i in range(len(iterable) - window_size + 1):
        yield iterable[i:i+window_size]

## Day 1: Sonar Sweep

1. How many measurements are larger than the previous measurement?
2. Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [40]:
in1 = data(1, parser=int)

In [41]:
def day1_1(depths):
    return quantify(sliding_window(depths, 2), lambda window: window[1] > window[0])

In [51]:
def day1_2(depths):
    sums = list(map(sum, sliding_window(depths, 3)))
    return quantify(sliding_window(sums, 2), lambda window: window[1] > window[0])

In [53]:
do(1, 1121, 1065)

[1121, 1065]

## Day 2: Dive!

1. What do you get if you multiply your final horizontal position by your final depth?

In [57]:
in2 = data(2, lambda line: line.split())

In [61]:
Position = tuple[int, int]

def move(position: Position, delta: Position) -> Position:
    return Position(map(operator.add, position, delta))

def navigate(instructions: list[str], position: Position = (0,0)) -> Position:
    for instruction in instructions:
        match instruction:
            case ["forward", n]:
                position = move(position, (int(n), 0))
            case ["down", n]:
                position = move(position, (0, int(n)))
            case ["up", n]:
                position = move(position, (0, -int(n)))
            case _:
                raise ValueError(f"Unmatched instruction: {instruction}")
    return position

In [69]:
def day2_1(instructions): return prod(navigate(instructions))

In [78]:
AimedPosition = namedtuple("AimedPosition", ["horizontal", "depth", "aim"])

def navigate_by_aim(instructions: list[str], position: AimedPosition = AimedPosition(0,0,0)) -> AimedPosition:
    for instruction in instructions:
        match instruction:
            case ["forward", n]:
                position = position._replace(
                    horizontal=position.horizontal + int(n),
                    depth=position.depth + int(n)*position.aim,
                )
            case ["down", n]:
                position = position._replace(aim=position.aim + int(n))
            case ["up", n]:
                position = position._replace(aim=position.aim - int(n))
            case _:
                raise ValueError(f"Unmatched instruction: {instruction}")
    return position

In [79]:
def day2_2(instructions):
    final_position = navigate_by_aim(instructions)
    return final_position.horizontal * final_position.depth

In [80]:
do(2, 1484118, 1463827010)

[1484118, 1463827010]